## A Hands-on Workshop series in Machine Learning
### Session 2: More on `pandas`: Groupby operations
#### Instructor: Aashita Kesarwani

##### Groupby operations
In the last session, we learned the basics of data manipulation using `pandas`. Today, we will learn the **split-apply-combine** operations by **grouping rows** of a dataframe

First we import the relevant python modules:

In [ ]:
import numpy as np
import pandas as pd

# The following two modules matplotlib and seaborn are for plots
import matplotlib.pyplot as plt
import seaborn as sns # Comment this if seaborn is not installed
%matplotlib inline

# The module re is for regular expressions
import re

Loading the [Titanic dataset from Kaggle](https://www.kaggle.com/c/titanic) stored in the `csv` file as a dataframe using [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) function.

In [ ]:
df = pd.read_csv('titanic.csv')

We use `head()` function to peek into the first 5 rows (or any number of rows by using `head(n)`).

In [ ]:
df.head()

[Description for the columns](https://www.kaggle.com/c/titanic/data) is as follows.  

|Variable|	Definition|	Key|   
|:---  |:--- |:---|
|PassengerId| Passenger ID |
|Survived| 	Survival|	0 = No, 1 = Yes |
|Pclass	|Ticket class|	1 = 1st, 2 = 2nd, 3 = 3rd|
|Sex	|Sex|	
|Age	|Age in years	|
|SibSp	|# of siblings / spouses aboard the Titanic	|
|Parch	|# of parents / children aboard the Titanic	|
|Ticket	|Ticket number	|
|Fare	|Passenger fare	|
|Cabin	|Cabin number	|
|Embarked	|Port of Embarkation	|C = Cherbourg, Q = Queenstown, S = Southampton|

Use the functions [`isnull()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isnull.html) and [`sum()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sum.html) on the dataframe to find out the number of missing values in each column.

In [ ]:
# df.isnull().sum()

Detecting missing values is an important step in Feature Engineering, that is preparing the features (independent variables) to use for building the machine learning models. The next step is to handle those missing values. 

1. If you consider filling the missing values, what are the possible options? 
2. Can you make use of other values in that column to fill the missing values? 
3. Can you make use of other values in that row as well as values in that column to fill the missing values?

As we saw earlier, the column *Age* has a lot of missing values. Those missing values can be filled with the median (or mean) age of all passengers.

* Can you think of other options to fill the missing age values for the passengers? 
* Does the title of the passengers' provide useful information about their age? 

In the exercise session in the last session, we created a new column *Title* that is derived from the column *Name* using regular expressions. Below is the compact repetition of the code to do the same.

In [ ]:
df['Title'] = df['Name'].apply(lambda name: re.findall("\w+[.]", name)[0])

df.Title.replace({'Ms.': 'Miss.', 'Mlle.': 'Miss.', 'Dr.': 'Rare', 'Mme.': 'Mrs.', 
                  'Major.': 'Rare', 'Lady.': 'Rare', 'Sir.': 'Rare', 'Col.': 'Rare', 
                  'Capt.': 'Rare', 'Countess.': 'Rare', 'Jonkheer.': 'Rare', 
                  'Dona.': 'Rare', 'Don.': 'Rare', 'Rev.': 'Rare'}, inplace=True)
df.head()

Let us make use of the newly derived *Title* column to fill the age based on the title of the passengers. For this, it would be helpful to group the passengers based on their title to get the median age for each group and then fill the missing age values for passengers with the median age for their title.

### 1. [GroupBy object](https://pandas.pydata.org/pandas-docs/version/0.22/groupby.html)

We can use [`groupby()`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.groupby.html) to group the rows of the dataframe based on a column(s), say *Title*.

In [ ]:
df.groupby('Title')

The resulting object is not a dataframe but **there are multiple ways to derive a dataframe from the grouped object**.

One of the ways to derive a dataframe from a groupby object is **by aggregation**, that is computing a summary statistic (or statistics) about each group. For example, we can get the median values for the columns in each group of titles.

In [ ]:
df.groupby('Title').median()

The median age vary greatly for each group ranging from 3.5 to 48 years.

In [ ]:
df.groupby('Title').mean()

The most common way to derive a dataframe from a groupby object is by transformation using [`transform()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transform.html).. This returns a dataframe with the same number of rows as the original dataframe. The `transform("median")` method operates on the numerical columns and replaces the original values with the median value for the corresponding group.

In [ ]:
df.groupby('Title').transform("median")

Our goal is to create a new column *MedianAge* which consists of the groupwise median age depending on the passengers' title. So, we use [`transform()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transform.html) only on the *Age* column instead of the whole dataframe.

In [ ]:
df.groupby('Title')['Age'].transform("median")

So far, our original dataframe has not really changed.

In [ ]:
df.head(3)

We create a new column *MedianAge* using [`groupby()`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.groupby.html) and [`transform()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transform.html) methods are demonstrated above.

In [ ]:
df['MedianAge'] = df.groupby('Title')['Age'].transform("median")
df.head(15)

Now we fill in the missing values in the *Age* column using the values in the *MedianAge* column.

In [ ]:
df['Age'] = df['Age'].fillna(df['MedianAge'])
df.head()

We drop off the *MedianAge* column since we no longer need it and then check for the missing values.

In [ ]:
df = df.drop('MedianAge', axis=1)
df.isnull().sum()

There are no missing values in the Age column!

In the next section, you will use the `groupby` object and create a new feature.

### 2. Feature engineering on the Titanic dataset to create a new column for group size 

In this exercise, we are going to create a new column called `GroupSize` to get the size of the group for each passenger. We consider groups to be either family members or those traveling on the same ticket. This feature is derived using three columns *SibSp*, *Parch* and *Ticket*. 

[Notes](https://www.kaggle.com/c/titanic/data) for the two features *SibSp* and *Parch* are as follows.

> SibSp: The dataset defines family relations in this way...  
Sibling = brother, sister, stepbrother, stepsister  
Spouse = husband, wife (mistresses and fiancés were ignored)
> 
> Parch: The dataset defines family relations in this way...  
Parent = mother, father  
Child = daughter, son, stepdaughter, stepson  
Some children travelled only with a nanny, therefore parch=0 for them.

Create a new column named *Family* by adding the columns *SibSp* and *Parch* and then add 1 to it. Hint: Use `df["New_column"] = df["column_1"] + df["column_2"] + 1`.

In [ ]:
# df['Family'] = df['SibSp'] + df['Parch'] + 1

Now we check the survival rates with respect to the family size.

In [ ]:
sns.barplot(x='Family', y='Survived', data=df);

Some passengers that appear to be traveling alone by account of their family size were part of a group traveling on the same ticket. To see this, get all the passengers traveling on the ticket "1601" (there are 7 of them).

In [ ]:
# df[df['Ticket']=='1601']

One can check that there are many tickets shared among passengers that may or may not be family members.

In [ ]:
df['Ticket'].value_counts()[:15]

Create a new column named *TicketCount* that counts the total number of passengers traveling in each passengers' ticket.

Hint: 
- First group passengers based on their tickets using `groupby()` on the *Ticket* column.
- For the grouped object, pick any column that has no missing values.
- Use `transform()` for this unique identifier column with the function `"count"` to create a new column *TicketCount*.

For example, we created *MedianAge* using the following code:   
```df['MedianAge'] = df.groupby('Title')['Age'].transform("median")```

In [ ]:
# df['TicketCount'] = df.groupby('Ticket')['Name'].transform("count")

Let us peek into the dataframe.

In [ ]:
df.head()

Plot the survival rates based on the *TicketCount* using [`sns.barplot()`](https://seaborn.pydata.org/generated/seaborn.barplot.html) (We used this function in the last session. Please refer to the notebook from session 1 if needed).

In [ ]:
# sns.barplot(x='TicketCount', y='Survived', data=df);

It does seem that the number of co-travelers have an impact on the survival rates.

Create a new column named *GroupSize* by picking the maximum value among the columns *Family* and *TicketCount*.   
Note: We consider groups to be either family members or those traveling on the same ticket.   
Hint: Use built-in [`max()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.max.html) function for pandas on the two relevant columns with the appropriate value for the `axis` parameter. 

In [ ]:
# df['GroupSize'] = df[['Family', 'TicketCount']].max(axis=1)


Plot the survival rates based on the GroupSize using [`sns.barplot()`](https://seaborn.pydata.org/generated/seaborn.barplot.html).

In [ ]:
# sns.barplot(x='GroupSize', y='Survived', data=df);

Check the number of rows where Groupsize is not equal to Family. Similarly, check the number of rows where TicketCount is not equal to Family.

In [ ]:
df[df['GroupSize'] != df['Family']].shape[0], df[df['GroupSize'] != df['TicketCount']].shape[0]

The output must be `(97, 191)`. Check your above code, if you get a different output.

#### Acknowledgment:
* [Titanic dataset from Kaggle](https://www.kaggle.com/c/titanic) dataset openly available in Kaggle is used in the exercises.
